In [ ]:
from pathlib import Path


ROOT = Path("../")
DATA = ROOT / "data"

In [ ]:
import os
import shutil


import cv2
from dvclive import Live
from ultralytics import YOLO

### Load data and split it into train/test

We have some [data in DVC](https://dvc.org/doc/start/data-management/data-versioning) that we can pull. 

This data includes:
* satellite images
* masks of the swimming pools in each satellite image

DVC can help connect your data to your repo, but it isn't necessary to have your data in DVC to start tracking experiments with DVC and DVCLive.

In [ ]:
!dvc pull

### Convert to YOLO Dataset format

https://docs.ultralytics.com/datasets/segment/

In [ ]:
def mask_to_yolo_annotation(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    annotation = ""
    for contour in contours:
        single_annotation = "0"
        for row, col in contour.squeeze():
            single_annotation += f" {round(col / mask.shape[1], 3)} {round(row / mask.shape[0], 3)}"
        annotation += f"{single_annotation}\n"
    return annotation

In [ ]:
test_regions = ["REGION_1-"]

train_data_dir = DATA / "yolo_dataset" / "train"
train_data_dir.mkdir(exist_ok=True, parents=True)
test_data_dir = DATA / "yolo_dataset" / "val"
test_data_dir.mkdir(exist_ok=True, parents=True)

for img_path in DATA.glob("pool_data/images/*.jpg"):
    yolo_annotation = mask_to_yolo_annotation(
        cv2.imread(
            str(DATA / "pool_data" / "masks" / f"{img_path.stem}.png"),
            cv2.IMREAD_GRAYSCALE
        )
    )

    if any(region in str(img_path) for region in test_regions):
        dst = test_data_dir / img_path.name
    else:
        dst = train_data_dir / img_path.name

    shutil.copy(img_path, dst)
    dst.with_suffix(".txt").write_text(yolo_annotation)

In [ ]:
yolo_dataset_yaml = DATA / "yolo_dataset.yaml"
yolo_dataset_yaml.write_text(
    """
path: ./yolo_dataset
train: train
val: val

names:
  0: swimming_pool
    """
)

### Train multiple models with different number of epochs
Set up model training, using DVCLive to capture the results of each experiment.

In [ ]:
def add_callbacks(live, yolo):
    def _log_confusion_matrix(validator, live):
        targets = []
        preds = []
        matrix = validator.confusion_matrix.matrix
        names = list(validator.names.values())
        if validator.confusion_matrix.task == "detect":
            names += ["background"]

        for ti, pred in enumerate(matrix.T.astype(int)):
            for pi, num in enumerate(pred):
                targets.extend([names[ti]] * num)
                preds.extend([names[pi]] * num)

        live.log_sklearn_plot("confusion_matrix", targets, preds)

    def on_train_epoch_start(trainer):
        trainer.__training_epoch = True

    def on_fit_epoch_end(trainer):
        if trainer.__training_epoch:
            all_metrics = {
                **trainer.label_loss_items(trainer.tloss, prefix="train"),
                **trainer.metrics,
            }
            for metric, value in all_metrics.items():
                live.log_metric(metric, value)

            live.next_step()
            trainer.__training_epoch = False

    def on_train_end(trainer):
        all_metrics = {
            **trainer.label_loss_items(trainer.tloss, prefix="train"),
            **trainer.metrics,
        }
        for metric, value in all_metrics.items():
            live.log_metric(metric, value, plot=False)

        _log_confusion_matrix(trainer.validator, live)

        for image_path in trainer.validator.plots.keys():
            if "val_batch" in image_path.name:
                live.log_image(image_path.name, image_path)

        if trainer.best.exists():
            live.log_artifact(
                trainer.best, name="pool-segmentation", type="model", copy=True,
                desc="This is a Computer Vision (CV) model that's segmenting out swimming pools from satellite images.",
                labels=["cv", "segmentation", "satellite-images", "yolo"],
            )

    yolo.callbacks["on_train_epoch_start"].append(on_train_epoch_start)
    yolo.callbacks["on_fit_epoch_end"].append(on_fit_epoch_end)
    yolo.callbacks["on_train_end"].append(on_train_end)

    return yolo

In [ ]:
imgsz = 384
epochs = 20
model = "yolov8n-seg.pt"
results_dir = ROOT / "results" / "train"

yolo = YOLO(model)

with Live(str(results_dir), save_dvc_exp=True, report=None, cache_images=True) as live:
    live.log_params({
        "epochs": epochs,
        "imgsz": imgsz,
        "model": model
    })
    yolo = add_callbacks(live, yolo)
    yolo.train(data=(DATA / "yolo_dataset.yaml").resolve(), epochs=epochs, imgsz=imgsz)

try:
    os.remove(DATA / "yolo_dataset" / "train.cache")
    os.remove(DATA / "yolo_dataset" / "val.cache")
except FileNotFoundError:
    pass

shutil.rmtree("../runs", ignore_errors=True)
shutil.rmtree("../weights", ignore_errors=True)
